In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
import pandas as pd
df = pd.read_csv('50_Startups.csv')

In [3]:
df.head()

,RND,ADMIN,MKT,STATE,PROFIT
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   RND     50 non-null     float64
 1   ADMIN   50 non-null     float64
 2   MKT     50 non-null     float64
 3   STATE   50 non-null     object 
 4   PROFIT  50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


In [5]:
s = df.isna().sum()
s

RND       0
ADMIN     0
MKT       0
STATE     0
PROFIT    0
dtype: int64

## Seperate X and Y features(Profit)

In [6]:
X = df.drop(columns=['PROFIT'])
Y = df[['PROFIT']]

In [7]:
X.head()

,RND,ADMIN,MKT,STATE
0,165349.20,136897.80,471784.10,New York
1,162597.70,151377.59,443898.53,California
2,153441.51,101145.55,407934.54,Florida
3,144372.41,118671.85,383199.62,New York
4,142107.34,91391.77,366168.42,Florida


In [8]:
Y.head()

,PROFIT
0,192261.83
1,191792.06
2,191050.39
3,182901.99
4,166187.94


## Seperate Cat and Con for X

In [9]:
cat = list(X.columns[X.dtypes=='object'])
con = list(X.columns[X.dtypes!='object'])

In [10]:
cat

['STATE']

In [11]:
con

['RND', 'ADMIN', 'MKT']

## Creating a preprocessing Pipeline

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [13]:
num_pipe = Pipeline(steps=[('impute', SimpleImputer(strategy='mean')),
                           ('scaler', StandardScaler())])

cat_pipe = Pipeline(steps=[('impute', SimpleImputer(strategy='most_frequent')),
                           ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

pre = ColumnTransformer([('num', num_pipe, con),
                         ('cat', cat_pipe, cat)]).set_output(transform='pandas')

In [14]:
x_pre = pre.fit_transform(X)
x_pre.head()

,num__RND,num__ADMIN,num__MKT,cat__STATE_California,cat__STATE_Florida,cat__STATE_New York
0,2.016411,0.560753,2.153943,0.0,0.0,1.0
1,1.955860,1.082807,1.923600,1.0,0.0,0.0
2,1.754364,-0.728257,1.626528,0.0,1.0,0.0
3,1.554784,-0.096365,1.422210,0.0,0.0,1.0
4,1.504937,-1.079919,1.281528,0.0,1.0,0.0


## Apply train and test Split

In [15]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x_pre, Y, test_size=0.2, random_state=21)


In [16]:
xtrain.head()

,num__RND,num__ADMIN,num__MKT,cat__STATE_California,cat__STATE_Florida,cat__STATE_New York
30,-0.258074,-0.205629,-0.990357,0.0,1.0,0.0
21,0.102724,1.169186,0.732788,0.0,0.0,1.0
19,0.279442,1.159837,-1.743127,0.0,0.0,1.0
11,0.593085,-1.065540,0.319834,1.0,0.0,0.0
22,0.006007,0.051850,0.762376,0.0,1.0,0.0


In [17]:
xtest.head()

,num__RND,num__ADMIN,num__MKT,cat__STATE_California,cat__STATE_Florida,cat__STATE_New York
7,1.245057,0.871980,0.932186,0.0,1.0,0.0
44,-1.134305,1.206419,-1.509074,1.0,0.0,0.0
43,-1.281134,0.217682,-1.449605,0.0,0.0,1.0
25,-0.199312,0.656489,-0.603517,1.0,0.0,0.0
14,1.017181,1.269199,0.375742,0.0,1.0,0.0


In [18]:
ytrain.head()

,PROFIT
30,99937.59
21,111313.02
19,122776.86
11,144259.40
22,110352.25


In [19]:
ytest.head()

,PROFIT
7,155752.60
44,65200.33
43,69758.98
25,107404.34
14,132602.65


In [20]:
xtrain.shape

(40, 6)

In [21]:
xtest.shape

(10, 6)

## Create a KNN Regression Model

In [22]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=10)
model.fit(xtrain, ytrain)

KNeighborsRegressor(n_neighbors=10)

In [23]:
model.score(xtrain, ytrain)

0.7313615687197589

In [24]:
model.score(xtest, ytest)

0.7531046602989124

## Hyperparameter Tuning with GridSearchCV

In [25]:
params = {"n_neighbors":[2,3,4,5,6,7,8,9,10]}

In [26]:
from sklearn.model_selection import GridSearchCV
kr = KNeighborsRegressor()
gscv = GridSearchCV(kr, param_grid=params, cv=5, scoring='neg_mean_absolute_error')
gscv.fit(xtrain, ytrain)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10]},
             scoring='neg_mean_absolute_error')

In [27]:
gscv.best_score_

-16171.048625

In [28]:
best_kr = gscv.best_estimator_
best_kr

KNeighborsRegressor(n_neighbors=2)

In [29]:
best_kr.score(xtrain, ytrain)

0.9106191272125944

In [30]:
best_kr.score(xtest, ytest)

0.8275361964808616

In [31]:
ypred_tr = best_kr.predict(xtrain)
ypred_ts = best_kr.predict(xtest)

In [32]:
ypred_tr[0:5]

array([[ 98358.255],
       [131762.395],
       [111890.75 ],
       [147009.68 ],
       [117309.575]])

In [33]:
ytrain.head()

,PROFIT
30,99937.59
21,111313.02
19,122776.86
11,144259.40
22,110352.25


In [34]:
ypred_ts[0:5]

array([[132926.21],
       [ 56054.62],
       [ 39803.74],
       [ 97070.32],
       [123659.53]])

In [35]:
ytest.head()

,PROFIT
7,155752.60
44,65200.33
43,69758.98
25,107404.34
14,132602.65


## Predicting out of sample data

In [38]:
xnew = pd.read_csv('test.csv')
xnew

,RND,ADMIN,MKT,STATE
0,78013.11,121597.55,264346.06,California
1,22177.74,154806.14,28334.72,California
2,23640.93,96189.63,148001.11,California
3,101913.08,110594.11,229160.95,Florida
4,77044.01,99281.34,140574.81,New York
5,542.05,51743.15,0.00,New York
6,61136.38,152701.92,88218.23,New York
7,100671.96,91790.61,249744.55,California
8,28663.76,127056.21,201126.82,Florida
9,20229.59,65947.93,185265.10,New York


In [39]:
xnew_pre = pre.transform(xnew)
xnew_pre

,num__RND,num__ADMIN,num__MKT,cat__STATE_California,cat__STATE_Florida,cat__STATE_New York
0,0.094441,0.009118,0.440446,1.0,0.0,0.0
1,-1.134305,1.206419,-1.509074,1.0,0.0,0.0
2,-1.102106,-0.906938,-0.520596,1.0,0.0,0.0
3,0.620398,-0.387599,0.149807,0.0,1.0,0.0
4,0.073115,-0.795469,-0.581939,0.0,0.0,1.0
5,-1.610433,-2.509409,-1.743127,0.0,0.0,1.0
6,-0.276958,1.130554,-1.014419,0.0,0.0,1.0
7,0.593085,-1.065540,0.319834,1.0,0.0,0.0
8,-0.991570,0.205925,-0.081763,0.0,1.0,0.0
9,-1.177178,-1.997270,-0.212785,0.0,0.0,1.0


In [40]:
pred = best_kr.predict(xnew_pre)
pred

array([[122733.48 ],
       [ 56054.62 ],
       [ 76252.125],
       [135194.425],
       [102515.775],
       [ 58451.235],
       [111890.75 ],
       [147009.68 ],
       [ 93743.555],
       [ 88854.285],
       [ 39803.74 ],
       [132926.21 ],
       [ 87288.875],
       [131762.395],
       [ 74869.2  ]])

In [41]:
xnew['PROFIT_PRED'] = pred

In [42]:
xnew

,RND,ADMIN,MKT,STATE,PROFIT_PRED
0,78013.11,121597.55,264346.06,California,122733.480
1,22177.74,154806.14,28334.72,California,56054.620
2,23640.93,96189.63,148001.11,California,76252.125
3,101913.08,110594.11,229160.95,Florida,135194.425
4,77044.01,99281.34,140574.81,New York,102515.775
5,542.05,51743.15,0.00,New York,58451.235
6,61136.38,152701.92,88218.23,New York,111890.750
7,100671.96,91790.61,249744.55,California,147009.680
8,28663.76,127056.21,201126.82,Florida,93743.555
9,20229.59,65947.93,185265.10,New York,88854.285


## Saving this result into CSV file

In [43]:
xnew.to_csv('Profit_Result.csv', index=False)